# Fine-tunning de BETO para predicción de Emojis

In [1]:
from config import *

Primero debemos importar el dataset. En nuestro caso este está guardado como un pandas pickleado (fuente: [este tutorial](https://huggingface.co/course/chapter5/2)). La biblioteca [datasets](https://huggingface.co/docs/datasets/index) nos permite cargar un dataset con este formato. Los path pueden ser modificados en el archivo config.py.

In [2]:
from datasets import load_dataset

# data_files = {"train": file_names['df_es_train'], "test": file_names['df_es_test'], "trial": file_names['df_es_trial']}
data_files = {"train": file_names['df_es_trial'], "test": file_names['df_es_test']}  # a modo de testeo no entrenaremos en el train set entero
dataset_emoji = load_dataset("pandas", data_files=data_files)
dataset_emoji

Using custom data configuration default-cb077f95137f42ce


Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 882.18it/s]


Dataset pandas downloaded and prepared to /home/camilo/.cache/huggingface/datasets/pandas/default-cb077f95137f42ce/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 526.10it/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 10000
    })
})

In [3]:
dataset_emoji["train"][0]

{'id': 'trial0',
 'text': 'Plaza de Oriente , Madrid .......#madrid #city #plazadeoriente #puertadesol #tour…',
 'label': '9'}